# Gaussian Naive Bayes Classifier

The aim of this project is to implement and test a Guassian Naive Bayes classifier.

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.metrics import f1_score, confusion_matrix, accuracy_score, classification_report, ConfusionMatrixDisplay
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
import timeit

%matplotlib inline

## GaussianNB Classifier
An implementation of a Gaussian Naive Bayes that fits the framework.

In [ ]:
class MyGaussianNB(BaseEstimator, ClassifierMixin):
    def fit(self, xt, yt):
        xt = np.array(xt)
        yt = np.array(yt)

        # first phase: compute class priors
        self._classes, self._class_priors = np.unique(yt, return_counts=True)
        self._class_priors = self._class_priors / float(np.sum(self._class_priors))  

        # second phase: compute all features' means and variances
        xt_split = [xt[yt==cl] for cl in self._classes]
        self._means = np.array([np.mean(xtcl, axis=0) for xtcl in xt_split])
        self._vars = np.array([np.var(xtcl, axis=0) for xtcl in xt_split])
        return self

    def predict(self, xtests):
        xtests = np.array(xtests)

        probas = self._compute_probas(xtests)
        return np.array([self._classes[i] for i in np.argmax(probas, axis=0)])

    def _compute_probas(self, xtests):
        return np.array([
            [
                self._class_priors[i] * np.product(self._conditional_proba(xtest, self._means[i], self._vars[i])) 
                for xtest in xtests
            ]
            for i in range(np.size(self._classes))
        ])
    
    @staticmethod
    def _conditional_proba(xt, m, var):
        return np.exp(-np.power(xt-m, 2)/(2*var))/np.sqrt(2*np.pi*var)

The function below will compare Scikit Gaussian NB and my implementation of Gaussian NB using accuracy and ROC analysis.

In [ ]:
def test_gaussian_on_dataset(x_train, y_train, x_test, y_test):
    X = np.concatenate((x_train, x_test))
    y = np.concatenate((y_train, y_test))
    gnb = GaussianNB()
    gnb.fit(x_train, y_train)
    y_pred = gnb.predict(x_test)
    cm = confusion_matrix(y_test, y_pred)
    report = classification_report(y_test, y_pred, target_names=['Not Helpful', 'Helpful'])
    disp = ConfusionMatrixDisplay(cm)
    fig, ax = plt.subplots(figsize=(3,3))
    ax.set_title("Scikit GNB Confusion Matrix")
    disp.plot(ax=ax)

    print(f"Scikit GNB - Report:\n{report}")
    print(f"Scikit GNB - Accuracy:\t{accuracy_score(y_test, y_pred)}")
    print(f"Scikit GNB - F1 Score:\t{f1_score(y_test, y_pred)}")
    print(f"Scikit GNB - 5x CV Accuracy:\t{cross_val_score(gnb, X, y).mean()}")
    print(f"Scikit GNB - 5x CV F1 Score:\t{cross_val_score(gnb, X, y, scoring='f1', error_score='raise').mean()}")
    print(f"Scikit GNB - 5x CV Precision:\t{cross_val_score(gnb, X, y, scoring='precision').mean()}")
    print(f"Scikit GNB - 5x CV Recall:\t{cross_val_score(gnb, X, y, scoring='recall').mean()}")

    print()
    
    mgnb = MyGaussianNB()
    mgnb.fit(x_train,y_train)
    y_pred = mgnb.predict(x_test)
    cm = confusion_matrix(y_test, y_pred)
    report = classification_report(y_test, y_pred, target_names=['Not Helpful', 'Helpful'])
    disp = ConfusionMatrixDisplay(cm)
    fig, ax = plt.subplots(figsize=(3,3))
    ax.set_title("My GNB Confusion Matrix")
    disp.plot(ax=ax)

    print(f"My GNB - Report:\n{report}")
    print(f"My GNB - Accuracy:\t{accuracy_score(y_test, y_pred)}")
    print(f"My GNB - F1 Score:\t{f1_score(y_test, y_pred)}")
    print(f"My GNB - 5x CV Accuracy:\t{cross_val_score(mgnb, X, y).mean()}")
    print(f"My GNB - 5x CV F1 Score:\t{cross_val_score(mgnb, X, y, scoring='f1').mean()}")
    print(f"My GNB - 5x CV Precision:\t{cross_val_score(mgnb, X, y, scoring='precision').mean()}")
    print(f"My GNB - 5x CV Recall:\t{cross_val_score(mgnb, X, y, scoring='recall').mean()}")

This function below will compare Scikit's Gaussian NB and mine in terms of speed.

In [ ]:
def test_speed_on_dataset(x_train, y_train, x_test, y_test, repeat=100):
    gnb = GaussianNB()
    gnb_fit_time = timeit.timeit(lambda: gnb.fit(x_train, y_train), number=repeat)
    gnb_predict_time = timeit.timeit(lambda: gnb.predict(x_test), number=repeat)
    
    print(f'Scikit GNB fit performance: {gnb_fit_time}s ({repeat} times)')
    print(f'Scikit GNB predict performance: {gnb_predict_time}s ({repeat} times)')

    mgnb = MyGaussianNB()
    mgnb_fit_time = timeit.timeit(lambda: mgnb.fit(x_train, y_train), number=repeat)
    mgnb_predict_time = timeit.timeit(lambda: mgnb.predict(x_test), number=repeat)

    print(f'My GNB fit performance: {mgnb_fit_time}s ({repeat} times)')
    print(f'My GNB predict performance: {mgnb_predict_time}s ({repeat} times)')

## Penguins dataset
Running and testing Gaussian Naive Bayes on the penguin dataset.

In [ ]:
penguins_af = pd.read_csv('penguins_af.csv', index_col = 0)
print(penguins_af.shape)
penguins_af.head()

In [ ]:
penguins_af['target'] = np.where(penguins_af['species'] == 'Adelie', 1, 0)
f_names = ['bill_length_mm', 'bill_depth_mm','flipper_length_mm', 'body_mass_g', 'species', 'target']
penguins = penguins_af[f_names]
penguins2C = penguins.loc[penguins['species'].isin(['Adelie','Chinstrap'])]
penguins2C.pop('species')
penguins2C.head()

In [ ]:
y = penguins2C.pop('target').values
X_raw = penguins2C.values
feature_names = penguins2C.columns
X_tr_raw, X_ts_raw, y_train, y_test = train_test_split(X_raw, y, test_size=1/2)
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_tr_raw)
X_test = scaler.transform(X_ts_raw)
X_train.shape, X_test.shape

In [ ]:
test_gaussian_on_dataset(X_train, y_train, X_test, y_test)

In [ ]:
test_speed_on_dataset(X_train, y_train, X_test, y_test)

## Diabetes dataset
Running and testing Gaussian Naive Bayes on the diabetes test dataset.

In [ ]:
diabetes = pd.read_csv('diabetes.csv', index_col = 0)
print(diabetes.shape)
diabetes.head()

In [ ]:
diabetes['target'] = np.where(diabetes['neg_pos'] == 'tested_positive', 1, 0)
diabetes.pop('neg_pos').values
diabetes.head()

In [ ]:
y = diabetes.pop('target').values
Xorig = diabetes.values
scaler = StandardScaler()
X = scaler.fit_transform(Xorig)
X.shape, y.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1/3)

In [ ]:
test_gaussian_on_dataset(X_train, y_train, X_test, y_test)

In [ ]:
test_speed_on_dataset(X_train, y_train, X_test, y_test)

## Hotel Reviews dataset
Running and testing Gaussian Naive Bayes on the helpfulness of hotel reviews dataset.

In [ ]:
reviews = pd.read_csv('HotelRevHelpfulness.csv', index_col=0)
print(reviews.shape)
reviews.head()

In [ ]:
y = reviews.pop('reviewHelpfulness').values
x_raw = reviews.values
scaler = StandardScaler()
x = scaler.fit_transform(x_raw)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=1/3)
x_train.shape, y_train.shape, x_test.shape, y_test.shape

In [ ]:
test_gaussian_on_dataset(x_train, y_train, x_test, y_test)

In [ ]:
test_speed_on_dataset(x_train, y_train, x_test, y_test)

# Conclusion

For binary classification:

- Evaluation: 

  - their confusion matrices are equals ;
  - their accuracies, f1 scores, recall and precisions are equals for the 3 datasets ;

- Speed:

  - `fit` method: Scikit is slower than mine (about 4 times) ;
  -  `predict` method: Scikit is faster than mine (about 40 times).

- What I suspect for these differences:

  - Scikit may compute other values when fitting the model in order to optimize the predictions ;
  - Scikit may use a better but equivalent algorithm to compute the conditional probabilities.